# PatchTST with Etherium (& more)

We will gather Etherium, Bitcoin, NASDAQ 100(^NDX), CBOE Interest Rate 10 Year T No (^TNX), KOSPI Composite Index (^KS11), Nikkei 225 (^N225)

In [ ]:
import yfinance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


